# Create_PMA_ontology_JSON
The purpose of this notebook is to create the ontology JSON file for the Princeton Mouse Atlas, just like what Allen provides for the AMBA. We will start from a CSV file containing the regions names, ids, and their parent:child relationships.

The ontology JSON file is needed for the merge ontology tool in Neuroglancer and is useful for cell counting at arbitrary levels of the ontology.

### Required:
- python >= 3.6
- numpy
- pandas 

### Optional:
- A system-wide installation of the command line program "graphviz". On OS X: "brew install graphviz". On linux: it might have been installed already. You can check with: dpkg -s graphivz. If it is not installed, you should be able to install it with your system's package manager. e.g. sudo apt-get install graphviz
- The graphviz python package, used to communicate between python and the lower level graphviz program: pip install graphviz

Graphviz will allow you to see a graph of the ontology, as in the last few cells in this notebook. If you don't want to use it, comment out the line "import graphviz" in the following code block.

In [2]:
import json
import numpy as np
import pandas as pd
import graphviz

In [3]:
# Read in csv file into pandas dataframe
df_filename = '/jukebox/LightSheetTransfer/atlas/Princeton_mouse_atlas_id_table_public.csv'
df_pma = pd.read_csv(df_filename)

In [4]:
df_pma

,name,acronym,id,parent_structure_id,parent_name,parent_acronym,voxels_in_structure
0,"Tuberomammillary nucleus, ventral part",TMv,1,557.0,Tuberomammillary nucleus,TM,13542
1,"Primary somatosensory area, mouth, layer 6b",SSp-m6b,2,345.0,"Primary somatosensory area, mouth",SSp-m,9621
2,secondary fissure,sec,3,1040.0,grooves of the cerebellar cortex,grv of CBX,0
3,Inferior colliculus,IC,4,339.0,"Midbrain, sensory related",MBsen,22705
4,internal capsule,int,6,784.0,corticospinal tract,cst,176771
...,...,...,...,...,...,...,...
1325,"Interpeduncular nucleus, intermediate",IPI,607344850,100.0,Interpeduncular nucleus,IPN,0
1326,"Interpeduncular nucleus, dorsomedial",IPDM,607344854,100.0,Interpeduncular nucleus,IPN,0
1327,"Interpeduncular nucleus, dorsolateral",IPDL,607344858,100.0,Interpeduncular nucleus,IPN,0
1328,"Interpeduncular nucleus, rostrolateral",IPRL,607344862,100.0,Interpeduncular nucleus,IPN,0


## Formatting for the ontology JSON file

Need to convert this structure into the ontology JSON structure.

Below is the structure of the Allen ontology JSON file so we can see the format we will need for the PMA:

In [5]:
allen_ontology_file = '/jukebox/LightSheetTransfer/atlas/allen_atlas/allen.json'

with open(allen_ontology_file) as json_file:
    allen_data = json.load(json_file)
allen_data

{'id': 997,
 'atlas_id': -1,
 'ontology_id': 1,
 'acronym': 'root',
 'name': 'root',
 'color_hex_triplet': 'FFFFFF',
 'graph_order': 0,
 'st_level': 0,
 'hemisphere_id': 3,
 'parent_structure_id': None,
 'children': [{'id': 8,
   'atlas_id': 0,
   'ontology_id': 1,
   'acronym': 'grey',
   'name': 'Basic cell groups and regions',
   'color_hex_triplet': 'BFDAE3',
   'graph_order': 1,
   'st_level': 1,
   'hemisphere_id': 3,
   'parent_structure_id': 997,
   'children': [{'id': 567,
     'atlas_id': 70,
     'ontology_id': 1,
     'acronym': 'CH',
     'name': 'Cerebrum',
     'color_hex_triplet': 'B0F0FF',
     'graph_order': 2,
     'st_level': 2,
     'hemisphere_id': 3,
     'parent_structure_id': 8,
     'children': [{'id': 688,
       'atlas_id': 85,
       'ontology_id': 1,
       'acronym': 'CTX',
       'name': 'Cerebral cortex',
       'color_hex_triplet': 'B0FFB8',
       'graph_order': 3,
       'st_level': 3,
       'hemisphere_id': 3,
       'parent_structure_id': 567,
   

The format is a nested dictionary, where each dictionary represents a brain region. The projenitor of all regions is the "root" region. The **children** key of each dictionary is a list of dictionaries, which themselves are child brain regions of the parent. These child dictionaries can also have **children** keys, which themselves  have **children** keys, and so on... until a child is reached with an emptry **children** list.

The only keys in each dictionary that we will need for the PMA are: **id, acronym, name, graph_order, parent_structure_id, children**

All elements have an integer **id**, string **acroynm**, string **name**.

All elements (i.e. individual dictionaries) except the root (id=997) have an integer **parent_structure_id**, root has a value of None since it has no parent.

All elements have a distinct integer **graph_order**, which starts at 0 with root and increases as you go deeper in hierarchy. Every parent must have a higher graph_order than each of its children. 

The **children** key is a list of dictionaries, where each dictionary has the same format as the root dictionary.

## Creating the ontology JSON file from the pandas dataframe

In [6]:
# Only keep necessary columns from the pandas dataframe 
df_pma = df_pma[['name','acronym','id','parent_structure_id']]
df_pma

,name,acronym,id,parent_structure_id
0,"Tuberomammillary nucleus, ventral part",TMv,1,557.0
1,"Primary somatosensory area, mouth, layer 6b",SSp-m6b,2,345.0
2,secondary fissure,sec,3,1040.0
3,Inferior colliculus,IC,4,339.0
4,internal capsule,int,6,784.0
...,...,...,...,...
1325,"Interpeduncular nucleus, intermediate",IPI,607344850,100.0
1326,"Interpeduncular nucleus, dorsomedial",IPDM,607344854,100.0
1327,"Interpeduncular nucleus, dorsolateral",IPDL,607344858,100.0
1328,"Interpeduncular nucleus, rostrolateral",IPRL,607344862,100.0


In [7]:
# Make a massive dictionary where keys are ids and values are dictionaries containing the
# info we will need in the ontology JSON file
# We will initialize each entry with an empty 'children' key which we will fill using the recursive function below
pma_df_dict = {}
for index, (name,acronym,ID,parentid) in df_pma.iterrows():
    if name != 'root': # we will handle the root dictionary separately below
        pma_df_dict[ID] = {'id':int(ID),'name':name,'acronym':acronym,
                           'parent_structure_id':int(parentid),'graph_order':0,'children':[]}


# intialize the graph_order global variable that we will increment as we add the children
graph_order = 0 

def make_ontology_dict(dic):
    """
    ---PURPOSE---
    A recursive function that makes a nested
    dictionary of the same format as the allen.json file,
    using the pma_df_dict we just made.
    ---INPUT---
    dic:    A parent dictionary to which you want to add child dictionaries
    """
    # Find all children of the current ID
    parent_id = dic['id']
    child_ids = [v['id'] for v in pma_df_dict.values() if v['parent_structure_id'] == parent_id]
    # add the correct graph_order value to this id
    global graph_order
    dic['graph_order'] = graph_order
    graph_order+=1 # increment so the next in line in the hierarchy has a higher graph_order
    """ Make list of child dicts to add to the entry in the ontology dictionary"""
    child_dicts = [pma_df_dict[ID] for ID in child_ids]
    dic['children']=child_dicts
    for child in child_dicts: 
        child_id = child['id']
        make_ontology_dict(child) # recursive call - child is a dictionary itself
    return
# Initialize the final dictionary which will contain the nested structure with the root id 
ontology_dic = {'id':997,'name':'root','graph_order':0,'parent_structure_id':None,'children':[]}
# Call the recursive function, using the intialized root dictionary as the starting point
make_ontology_dict(dic=ontology_dic)

In [8]:
# Let's see how it came out:
ontology_dic

{'id': 997,
 'name': 'root',
 'graph_order': 0,
 'parent_structure_id': None,
 'children': [{'id': 8,
   'name': 'Basic cell groups and regions',
   'acronym': 'grey',
   'parent_structure_id': 997,
   'graph_order': 1,
   'children': [{'id': 343,
     'name': 'Brain stem',
     'acronym': 'BS',
     'parent_structure_id': 8,
     'graph_order': 2,
     'children': [{'id': 313,
       'name': 'Midbrain',
       'acronym': 'MB',
       'parent_structure_id': 343,
       'graph_order': 3,
       'children': [{'id': 323,
         'name': 'Midbrain, motor related',
         'acronym': 'MBmot',
         'parent_structure_id': 313,
         'graph_order': 4,
         'children': [{'id': 35,
           'name': 'Oculomotor nucleus',
           'acronym': 'III',
           'parent_structure_id': 323,
           'graph_order': 5,
           'children': []},
          {'id': 58,
           'name': 'Medial terminal nucleus of the accessory optic tract',
           'acronym': 'MT',
           'pare

In [14]:
# Now write this to a JSON file
PMA_json_filename = './PMA_ontology.json'
with open(PMA_json_filename,'w') as outfile:
    json.dump(ontology_dic,outfile,indent=2) # indent is used to make formatting identical to allen.json

## (Optional) Visualize the ontology dictionary as a directed graph

Now make a visual graph of this to compare to Allen graph to make sure it is correct

In [10]:
""" Make the graph that will be used
in the get_parent() function below """

def make_id_graph(dic,graph):
    """ Make a edge-unweighted directed graph from a dictionary
    Representing a brain ontology
    """
    name = dic.get('name')
    acronym = dic.get('acronym')
    children = dic.get('children')
    orig_id = dic.get('id')
    graph.node(name,f'{acronym}: {orig_id}')
    for child in children:
        child_name = child.get('name')
        graph.edge(name,child_name)
        make_id_graph(child,graph)
    return 

In [11]:
# First Allen Mouse Brain Atlas
G_Allen = graphviz.Digraph()
make_id_graph(allen_data,G_Allen)
G_Allen.save("Allen_graph.dot")

'Allen_graph.dot'

In [12]:
# Now Princeton Mouse Brain Atlas:
G_PMA = graphviz.Digraph()
make_id_graph(ontology_dic,G_PMA)
G_PMA.save("PMA_graph.dot")

'PMA_graph.dot'

These files can be viewed in this notebook, e.g.: 

In [13]:
G_PMA

But because they are such large graphs, it is easier to make a pdf from them and then view them in the browser 
They can be converted to PDF from the command line using the program "dot" which gets installed with graphviz, e.g.:
```
$ dot -Tpdf PMA_graph.dot -o PMA_graph.pdf
```
Then open the PMA_graph.pdf file in a browser and zoom way in until you can read the nodes of the graph.

For example, copy this link into your browser to view the PMA ontology saved on bucket:

```
file:///jukebox/LightSheetTransfer/atlas/PMA_graph.pdf
```
You will need to zoom in a lot to see the nodes.